In [ ]:
!which python

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
# basic packages
import os
import re
import sys
import datetime
from typing import List, Dict, Tuple, Optional, Any
from itertools import combinations, product
from pathlib import Path
import glob
#import yaml
import tqdm
import multiprocessing as mp

In [ ]:
# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# bioinformatics
from bintools.utils.utils import get_yaml_config

In [ ]:
ROOT_dir = Path(os.path.abspath(os.path.join(Path("../")))).__str__()
if ROOT_dir not in sys.path:
    sys.path.append(ROOT_dir)

In [ ]:
def sign(x):
     return np.sum(x >= 0.95) / x.shape[0] * 100
     
def prop(x):
     return np.sum(x) / x.shape[0]

def tran(x):
    if x <= 1:
        return 0
    else:
        return 1


def concat(input_dir:str, pattern:str)-> pd.DataFrame:
     files: List[str] = glob.glob(input_dir + pattern)
     assert len(files) > 0
     list_of_df : List[pd.DataFrame] = []
     for f in files:
          cur_df: pd.DataFrame = pd.read_csv(f,sep="\t")
          list_of_df += [cur_df]
     return pd.concat(list_of_df,axis=0,ignore_index=True)

In [ ]:
list_of_geneID_simu: List[str] = get_yaml_config(ROOT_dir+"/configs/configs.yaml")["simulation"]["geneID"]
list_of_geneID_emp: List[str] = get_yaml_config(ROOT_dir+"/configs/configs.yaml")["empirical"]["geneID"]

## PPRED 

### Empirical study

#### GTR+G

In [ ]:
list_of_df_ppred = []
modelID = "GTR4G"
repID = "A"
for geneID in list_of_geneID_emp:
    try:
        list_of_df_ppred += [pd.read_csv(ROOT_dir+"/outputs/empirical/stats/{modelID}/{geneID}-{repID}_ppred.tsv".format(modelID=modelID,geneID=geneID,repID=repID),sep="\t",index_col=0)]
    except Exception as e:
        print("something wrong with %s"% geneID)
df_ppred_concat = pd.concat(list_of_df_ppred)

In [ ]:
assert len(set(df_ppred_concat["geneID"])) == 137 

In [ ]:
dict_of_stats = {}
k=0
for geneID in list(set(df_ppred_concat.geneID)):
    dict_of_stats[k] = {
            "geneID": geneID,
        }
    for x, y in list(product(["A", "C", "G", "T"], ["A", "C", "G", "T"])):
        XpY = x + "p" + y
        XpY_ppred: np.array = df_ppred_concat.loc[(df_ppred_concat["geneID"]==geneID)][XpY].to_numpy()
        XpY_obs: float = df_obs_concat.loc[df_obs_concat["geneID"]==geneID][XpY].to_numpy()[0]
        
        dict_of_stats[k].update({
                XpY+"_test" : np.sum(XpY_ppred > XpY_obs)/len(XpY_ppred),
                XpY+"_mean" : np.mean(XpY_ppred),
                XpY+"_std" : np.std(XpY_ppred),
                XpY+"_obs": XpY_obs,
                })
        
    k+=1

In [ ]:
sign(pd.DataFrame.from_dict(data=dict_of_stats,orient="index")[["TpA_test"]])

In [ ]:
pd.DataFrame.from_dict(data=dict_of_stats,orient="index")[["CpG_test"]].agg([sign]).sort_values(by=["sign"],axis=1).round(3).to_csv(ROOT_dir + "/reports/CpG_test_empirical_GTRG.csv", sep="\t")

In [ ]:
pd.DataFrame.from_dict(data=dict_of_stats,orient="index")[[x+"p"+y+"_test"for x, y in list(product(["A", "C", "G", "T"], ["A", "C", "G", "T"]))]].agg([sign]).sort_values(by=["sign"],axis=1).round(3).to_csv(ROOT_dir + "/reports/XpY_test_empirical_GTRG.csv", sep="\t")

In [ ]:
pd.DataFrame.from_dict(data=dict_of_stats,orient="index")[["geneID"]+[x+"p"+y+"_test"for x, y in list(product(["A", "C", "G", "T"], ["A", "C", "G", "T"]))]].to_csv(ROOT_dir + "/reports/XpY_test_empirical_all_GTRG.csv", sep="\t")

In [ ]:
fig, axes = plt.subplots(1,2,sharex=True, sharey="row", figsize=(6,4))
axes = axes.ravel()
list_of_subplots = ["A","B"]
k = 0
obs = df_obs_concat.loc[df_obs_concat["geneID"]== "MEP1A",["CpG"]].values[0][0]
pred = df_ppred_concat.loc[(df_ppred_concat["geneID"]=="MEP1A"),["CpG"]].values.reshape(1,-1)[0]
weights_pred = np.ones_like(pred) / pred.shape[0]
v_ratio = str(round(np.sum(obs > pred) / obs.size * 100, 2))
bins = np.histogram(
                    np.hstack([pred]), bins=20
                )[1]
axes[k].set_title(list_of_subplots[k], loc="left")
axes[k].set_xlabel("CpG frequency")
axes[k].hist(
    [pred],
    bins=bins,
    color="blue",
    alpha=0.5,
    stacked=False,
    weights=[weights_pred],
    label=["% " + v_ratio],
)
# plt.axis('off')
# plt.yaxis().set_visible(False)
axes[k].axvline(obs, color="black")
# _ = axes.set_yticks([])
# _ = axes.set_yticklabels([])
axes[k].set_ylabel("Density")

k +=1
obs = df_obs_concat.loc[df_obs_concat["geneID"]== "MEP1A",["ApT"]].values[0][0]
pred = df_ppred_concat.loc[(df_ppred_concat["geneID"]=="MEP1A"),["ApT"]].values.reshape(1,-1)[0]
weights_pred = np.ones_like(pred) / pred.shape[0]
v_ratio = str(round(np.sum(obs > pred) / obs.size * 100, 2))
bins = np.histogram(
                    np.hstack([pred]), bins=20
                )[1]
axes[k].set_title(list_of_subplots[k], loc="left")
axes[k].set_xlabel("ApT frequency")
axes[k].hist(
    [pred],
    bins=bins,
    color="blue",
    alpha=0.5,
    stacked=False,
    weights=[weights_pred],
    label=["% " + v_ratio],
)
# plt.axis('off')
# plt.yaxis().set_visible(False)
axes[k].axvline(obs, color="black")


fig.savefig(ROOT_dir + "/reports/figure1.pdf",dpi=300)

#### F1X4

In [ ]:
modelID = "F1X4"
repID = "A"
list_of_df_ppred = []
for geneID in list_of_geneID_emp:
    try:
        list_of_df_ppred += [pd.read_csv(ROOT_dir+"/outputs/empirical/{modelID}/stats/{geneID}-{repID}_ppred.tsv".format(modelID=modelID,geneID=geneID,repID=repID),sep="\t",index_col=0)]
    except Exception as e:
        print("something wrong with %s"% geneID)
df_ppred_concat = pd.concat(list_of_df_ppred)

In [ ]:
assert len(set(df_ppred_concat["geneID"])) == 137 

In [ ]:
dict_of_stats = {}
k=0
for geneID in list(set(df_ppred_concat.geneID)):
    dict_of_stats[k] = {
            "geneID": geneID,
        }
    for x, y in list(product(["A", "C", "G", "T"], ["A", "C", "G", "T"])):
        XpY = x + "p" + y
        XpY_ppred: np.array = df_ppred_concat.loc[(df_ppred_concat["geneID"]==geneID)][XpY].to_numpy()
        XpY_obs: float = df_obs_concat.loc[df_obs_concat["geneID"]==geneID][XpY].to_numpy()[0]
        
        dict_of_stats[k].update({
                XpY+"_test" : np.sum(XpY_ppred > XpY_obs)/len(XpY_ppred),
                XpY+"_mean" : np.mean(XpY_ppred),
                XpY+"_std" : np.std(XpY_ppred),
                XpY+"_obs": XpY_obs,
                })
        
    k+=1

In [ ]:
pd.DataFrame.from_dict(data=dict_of_stats,orient="index")[[x+"p"+y+"_test"for x, y in list(product(["A", "C", "G", "T"], ["A", "C", "G", "T"]))]].agg([np.mean]).sort_values(by=["mean"],axis=1).round(3).to_csv(ROOT_dir + "/reports/XpY_test_GTRG.csv", sep="\t")

### Simulation study

#### GTR4G on simulations generated using GTR4G

In [ ]:
modelID = "GTR4G"
repID = "A"
list_of_df_obs = []
for geneID in list_of_geneID_simu:
    for drawID in range(0,10):
        cur_df = pd.read_csv(ROOT_dir+"/outputs/simulation/{modelID}/simu/stats/{geneID}-{repID}-{draw}-OBSERVED.tsv".format(modelID=modelID,geneID=geneID,repID=repID,draw=drawID),sep="\t", index_col=0)
        list_of_df_obs += [cur_df]

df_obs_concat = pd.concat(list_of_df_obs, ignore_index=True)

In [ ]:
modelID = "GTR4G"
repID = "A"
repID_ = "A"

list_of_df_ppred = []
for geneID in list_of_geneID_simu:
    for drawID in range(0,10):
        cur_df = pd.read_csv(ROOT_dir+"/outputs/simulation/{modelID}/simu/stats/{geneID}-{repID}-{draw}-{repID_}_ppred.tsv"
        ,sep="\t", index_col=0)
        list_of_df_ppred += [cur_df]
df_ppred_concat = pd.concat(list_of_df_ppred, ignore_index=True)

In [ ]:
dict_of_stats = {}
k=0
for geneID in list_of_geneID_simu:
    for drawID in range(0,10):
        dict_of_stats[k] = {
            "geneID": geneID,
            "drawID":drawID
        }
        for x, y in list(product(["A", "C", "G", "T"], ["A", "C", "G", "T"])):
            XpY = x + "p" + y

            XpY_ppred: np.array = df_ppred_concat.loc[(df_ppred_concat["geneID"]==geneID)&(df_ppred_concat["draw"]==drawID)][XpY].to_numpy()
            XpY_obs: float = df_obs_concat.loc[(df_obs_concat["geneID"]==geneID)&(df_obs_concat["draw"]==drawID)][XpY].to_numpy()[0]
            dict_of_stats[k].update({
                XpY+"_test" : np.sum(XpY_ppred > XpY_obs)/len(XpY_ppred),
                XpY+"_mean" : np.mean(XpY_ppred),
                XpY+"_std" : np.std(XpY_ppred),
                XpY+"_obs":XpY_obs,
            })
        k+=1

In [ ]:
pd.DataFrame.from_dict(data=dict_of_stats,orient="index")[["CpG_test"]].agg([sign, "count"]).round(2)#.to_csv(ROOT_dir + "/reports/CpG_test_simu_GTRG_GTRG_.csv", sep="\t")

In [ ]:
pd.DataFrame.from_dict(data=dict_of_stats,orient="index")[["CpG_test"]].agg([sign, "count"])#[["CpG_test","CpG_mean","CpG_std","CpG_obs"]]

#### GTR4G on simulations generated using F1X4

In [ ]:
modelID = "F1X4"
repID = "A"
list_of_df_obs = []
for geneID in list_of_geneID_simu:
    for omega in [0.2,1]:
        for CpG in [1,4,8]:
            for TpA in [1,]:
                for tbl in [1,10]:
                    print(".",end="")
                    for drawID in range(0,10):
                        cur_df = pd.read_csv(ROOT_dir+"/outputs/simulation/{modelID}/simu/stats/{geneID}-{repID}-{omega}-{CpG}-{TpA}-{tbl}-{draw}-1_0-OBSERVED.tsv".format(modelID=modelID,geneID=geneID,repID=repID,omega=omega,CpG=CpG,TpA=TpA,draw=drawID),sep="\t", index_col=0)
                        cur_df["tbl"] = [tbl]*cur_df.shape[0]
                        list_of_df_obs += [cur_df]

df_obs_concat = pd.concat(list_of_df_obs, ignore_index=True)

In [ ]:
modelID = "F1X4"
repID = "A"
repID_ = "A"
list_of_df_ppred = []
for geneID in list_of_geneID_simu:
    for omega in [0.2,1]:
        for CpG in [1,4,8]:
            for TpA in [1]:
                for tbl in [1]:
                    print(".",end="")
                    for drawID in range(0,10):
                        cur_df = pd.read_csv(ROOT_dir+"/outputs/simulation/{modelID}/simu/stats/{geneID}-{repID}-{omega}-{CpG}-{TpA}-{tbl}-{drawID}-{repID}_ppred.tsv".format(modelID=modelID,geneID=geneID,repID=repID,omega=omega,CpG=CpG,TpA=TpA,drawID=drawID,repID_=repID_),sep="\t", index_col=0)
                        cur_df["tbl"] = [tbl]*cur_df.shape[0]
                        list_of_df_ppred += [cur_df]
df_ppred_concat = pd.concat(list_of_df_ppred, ignore_index=True)

In [ ]:
dict_of_stats = {}
k=0
for geneID in list_of_geneID_simu:
    for omega in [0.2,1]:
        for CpG in [1,4,8]:
            for TpA in [1,]:
                for tbl in [1]:
                    for drawID in range(0,10):
                        dict_of_stats[k] = {
                            "geneID": geneID,
                            "CpG": CpG,
                            "TpA": TpA,
                            "tbl": tbl,
                            "omega":omega,
                            "drawID":drawID
                        }
                        for x, y in list(product(["A", "C", "G", "T"], ["A", "C", "G", "T"])):
                            XpY = x + "p" + y

                            XpY_ppred: np.array = df_ppred_concat.loc[(df_ppred_concat["tbl"]==tbl)&(df_ppred_concat["geneID"]==geneID)&(df_ppred_concat["omega"]==omega)&(df_ppred_concat["CpGf"]==CpG)&(df_ppred_concat["TpAf"]==TpA)&(df_ppred_concat["draw"]==drawID)][XpY].to_numpy()
                            XpY_obs: float = df_obs_concat.loc[(df_obs_concat["tbl"]==tbl)&(df_obs_concat["geneID"]==geneID)&(df_obs_concat["omega"]==omega)&(df_obs_concat["CpGf"]==CpG)&(df_obs_concat["TpAf"]==TpA)&(df_obs_concat["draw"]==drawID)][XpY].to_numpy()[0]
                            dict_of_stats[k].update({
                                XpY+"_test" : np.sum(XpY_ppred > XpY_obs)/len(XpY_ppred),
                                XpY+"_mean" : np.mean(XpY_ppred),
                                XpY+"_std" : np.std(XpY_ppred),
                                XpY+"_obs":XpY_obs,
                            })
                        k+=1

In [ ]:
pd.DataFrame.from_dict(data=dict_of_stats,orient="index").groupby(by=["CpG","TpA","tbl","omega"])[["CpG_test"]].agg([sign, "count"]).round(2).to_csv(ROOT_dir + "/reports/CpG_test_simu_GTR4G_on_F1X4.csv", sep="\t")